# running EC2 commands via boto3

Boto3 SSM documentation:
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ssm.html#SSM.Client.send_command

AWS EC2 instances:
https://console.aws.amazon.com/ec2/v2/home?region=us-east-1#Instances:

Couple of gotchas while POCing:
  * required install of SSM on EC2: https://docs.aws.amazon.com/systems-manager/latest/userguide/agent-install-al2.html
  * required IAM permissions for SSM: https://docs.aws.amazon.com/systems-manager/latest/userguide/setup-instance-profile.html
  * seemed to require a reboot

In [2]:
import time
import boto3

In [3]:
# init boto3 SSM client
client = boto3.client('ssm', region_name='us-east-1')

In [5]:
# send hello world
results = client.send_command(
    InstanceIds=["i-07a0c73b0ceb6ce87"],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": ["echo 'hello world'"]},
)
print(results)

{'Command': {'CommandId': 'd1f30d58-6f64-4bac-a02a-7d47d57bc399', 'DocumentName': 'AWS-RunShellScript', 'DocumentVersion': '$DEFAULT', 'Comment': '', 'ExpiresAfter': datetime.datetime(2022, 1, 15, 12, 10, 22, 624000, tzinfo=tzlocal()), 'Parameters': {'commands': ["echo 'hello world'"]}, 'InstanceIds': ['i-07a0c73b0ceb6ce87'], 'Targets': [], 'RequestedDateTime': datetime.datetime(2022, 1, 15, 10, 10, 22, 624000, tzinfo=tzlocal()), 'Status': 'Pending', 'StatusDetails': 'Pending', 'OutputS3Region': 'us-east-1', 'OutputS3BucketName': '', 'OutputS3KeyPrefix': '', 'MaxConcurrency': '50', 'MaxErrors': '0', 'TargetCount': 1, 'CompletedCount': 0, 'ErrorCount': 0, 'DeliveryTimedOutCount': 0, 'ServiceRole': '', 'NotificationConfig': {'NotificationArn': '', 'NotificationEvents': [], 'NotificationType': ''}, 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}, 'TimeoutSeconds': 3600}, 'ResponseMetadata': {'RequestId': 'd48f1431-0068-4935-b06f-f73b1e6801d1', 'H

In [ ]:
results

In [ ]:
# get results of command (noting this won't be needed mostly)
results = client.get_command_invocation(
    InstanceId="i-07a0c73b0ceb6ce87",
    CommandId="4c43e368-523f-4c06-a35e-e4a4de27b857"
)

In [ ]:
results

## notes on using SSM for issuing Kafka CLI commands on EC2 

First pass at running Kafka CLI command failed due to following permission error:

```
An error occurred (AccessDeniedException) when calling the DescribeCluster operation: User: arn:aws:sts::007385147054:assumed-role/SSMInstanceProfile/i-07a0c73b0ceb6ce87 is not authorized to perform: kafka:DescribeCluster on resource: arn:aws:kafka:us-east-1:007385147054:cluster/poc-cluster-1/7a7a5d89-0d30-4a3c-94ab-b6c4f18c0fc0-21\nfailed to run commands: exit status 255
```

This was resolved by granting more IAM permissions to the profile -- `arn:aws:iam::007385147054:role/SSMInstanceProfile` -- that was granted to the ECS machine that allowed SSM commands. 

In [ ]:
# run Kafka CLI command; polling for results
# prepare command string
cmd = 'aws kafka describe-cluster --region us-east-1 --cluster-arn "arn:aws:kafka:us-east-1:007385147054:cluster/poc-cluster-1/7a7a5d89-0d30-4a3c-94ab-b6c4f18c0fc0-21"'

# send command
cmd_send_results = client.send_command(
    InstanceIds=["i-07a0c73b0ceb6ce87"],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": [cmd]},
)

# wait 2 seconds (improve w/ polling?)
time.sleep(2)

# retrieve results of SSM command and print stdout
cmd_get_results = client.get_command_invocation(
    InstanceId="i-07a0c73b0ceb6ce87",
    CommandId=cmd_send_results['Command']['CommandId']
)
print(cmd_get_results['StandardOutputContent'])

In [6]:
# testing speed of sending SSM messages

def test_ssm_cmds():
    t0 = time.time()
    for x in range(0,10):
        t1 = time.time()
        results = client.send_command(
            InstanceIds=["i-07a0c73b0ceb6ce87"],
            DocumentName="AWS-RunShellScript",
            Parameters={"commands": ["echo 'hello world'"]},
        )
        print(f"{x} elapsed: {time.time()-t1}")
    print(f"total elapsed: {time.time()-t0}")

test_ssm_cmds()

0 elapsed: 0.6463830471038818
1 elapsed: 0.21069979667663574
2 elapsed: 0.26608896255493164
3 elapsed: 0.2220611572265625
4 elapsed: 0.2794616222381592
5 elapsed: 0.23826336860656738
6 elapsed: 0.16964292526245117
7 elapsed: 0.31514930725097656
8 elapsed: 0.24797296524047852
9 elapsed: 0.25336503982543945
total elapsed: 2.8519375324249268
